In [61]:
import pandas as pd
import numpy as np
import os
import csv
import shutil
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers.legacy import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [62]:
# Define the directory paths
dir_paths = ["Dataset/ECG Images of COVID-19 Patients (250)", "Dataset/ECG Images of Myocardial Infarction Patients (77)", "Dataset/ECG Images of Patient that have abnormal heart beats (548)", "Dataset/ECG Images of Patient that have History of MI (203)", "Dataset/Normal Person ECG Images (859)"]

# Create a list to store the image filenames
image_filenames = []

# Loop through the directories and add the image filenames to the list
for dir_path in dir_paths:
    for filename in os.listdir(dir_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_filenames.append(filename)

# Create a CSV file and write the image filenames and label "0" to it
with open("image_labels.csv", mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["filename", "label"])
    for image_filename in image_filenames:
        if "Normal" in image_filename:
            writer.writerow([image_filename, "0"])
        else:
            writer.writerow([image_filename, "1"])

In [63]:
import os
import cv2

# directory containing the images to be cropped
input_dir = "Dataset/ECG Images of COVID-19 Patients (250)/"

# loop through all files in the directory
for filename in os.listdir(input_dir):
    # check if the file is an image file
    if filename.endswith(".jpg"):
        # read the image
        img = cv2.imread(os.path.join(input_dir, filename))
        
        # perform the cropping operation
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
        morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)
        cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
        cnt = sorted(cnts, key=cv2.contourArea)[-1]
        x,y,w,h = cv2.boundingRect(cnt)
        dst = img[y:y+h, x:x+w]
        
        # save the cropped image with the same filename
        cv2.imwrite(os.path.join(input_dir, filename), dst)


In [64]:
# define paths
source_folder = 'Dataset'
destination_folder = 'images'

# create destination folder if not exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# loop through subdirectories
for subdir in os.listdir(source_folder):
    subdir_path = os.path.join(source_folder, subdir)
    
    # check if it is a directory and not a file
    if os.path.isdir(subdir_path) and "ECG" in subdir_path:
        print(subdir)
        
        # loop through files in subdirectory
        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)
            
            # copy file to destination folder
            shutil.copy(file_path, destination_folder)

ECG Images of COVID-19 Patients (250)
ECG Images of Patient that have History of MI (203)
ECG Images of Patient that have abnormal heart beats (548)
Normal Person ECG Images (859)
ECG Images of Myocardial Infarction Patients (77)


In [65]:
# Set random seed for reproducibility
np.random.seed(42)

# Load CSV file containing image filenames and labels
df = pd.read_csv('image_labels.csv')
df["label"] = df["label"].astype(str)

In [66]:
# Split dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

In [67]:
# Create directories for train, validation, and test sets
train_dir = 'train'
os.makedirs(train_dir, exist_ok=True)

test_dir = 'test'
os.makedirs(test_dir, exist_ok=True)

In [68]:
# Move images to respective directories based on the split
for i, row in train_df.iterrows():
    src = 'images/' + row['filename']
    dst = train_dir + '/' + row['filename']
    shutil.copyfile(src, dst)

for i, row in test_df.iterrows():
    src = 'images/' + row['filename']
    dst = test_dir + '/' + row['filename']
    shutil.copyfile(src, dst)

In [69]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   preprocessing_function=preprocess_input)

# No data augmentation for testing set, only rescaling
test_datagen = ImageDataGenerator(rescale=1./255,
                                  preprocessing_function=preprocess_input)


In [70]:
# Define batch size
batch_size = 32

# Set target image size
target_size = (224, 224)

In [71]:
# Create training and testing generators
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    directory=train_dir,
                                                    x_col='filename',
                                                    y_col='label',
                                                    target_size=target_size,
                                                    batch_size=batch_size,
                                                    class_mode='binary')

test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                                  directory=test_dir,
                                                  x_col='filename',
                                                  y_col='label',
                                                  target_size=target_size,
                                                  batch_size=batch_size,
                                                  class_mode='binary')

Found 1541 validated image filenames belonging to 2 classes.
Found 386 validated image filenames belonging to 2 classes.


In [72]:
# Build CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [73]:
# Compile model
#KerasClassifier(model = model, optimizer=tf.keras.optimizers.Adam(), epochs=100, verbose=0)
from tensorflow import keras
adam = keras.optimizers.Adam()

model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5)
best_model = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

In [74]:
# Train model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=20,
                    validation_data=test_generator,
                    validation_steps=len(test_generator),
                    callbacks=[early_stop, best_model])

Epoch 1/20
49/49 [==============================] - 55s 1s/step - loss: 0.7012 - accuracy: 0.5490 - val_loss: 0.6838 - val_accuracy: 0.5544
Epoch 2/20
49/49 [==============================] - 57s 1s/step - loss: 0.6655 - accuracy: 0.5548 - val_loss: 0.9217 - val_accuracy: 0.5544
Epoch 3/20
49/49 [==============================] - 57s 1s/step - loss: 0.6277 - accuracy: 0.6080 - val_loss: 0.5722 - val_accuracy: 0.6244
Epoch 4/20
49/49 [==============================] - 59s 1s/step - loss: 0.5775 - accuracy: 0.6671 - val_loss: 1.7707 - val_accuracy: 0.5389
Epoch 5/20
49/49 [==============================] - 62s 1s/step - loss: 0.5368 - accuracy: 0.6885 - val_loss: 0.8863 - val_accuracy: 0.5829
Epoch 6/20
49/49 [==============================] - 60s 1s/step - loss: 0.5311 - accuracy: 0.7151 - val_loss: 0.5177 - val_accuracy: 0.7461
Epoch 7/20
49/49 [==============================] - 60s 1s/step - loss: 0.4960 - accuracy: 0.7313 - val_loss: 2.7318 - val_accuracy: 0.5440
Epoch 8/20
49/49 [==

In [56]:
# Evaluate model on testing set
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)

# Save model
model.save('my_model.h5')

# Save training history to CSV file
history_df = pd.DataFrame(history.history)
history_df.to_csv('training_history.csv', index=False)

13/13 [==============================] - 9s 644ms/step - loss: 0.7411 - accuracy: 0.6736
Test accuracy: 0.6735751032829285


In [ ]:
y_pred = model.predict(X_test)
y_pred_binary = np.round(y_pred) 

In [80]:
import cv2
import os
import tensorflow as tf
import numpy as np

# define the categories
categories = ['Low Risk', 'High Risk']

# define the path to the directory containing images
img_dir = 'Test/'

# loop through the images and make predictions
for img_name in os.listdir(img_dir):
    img_path = os.path.join(img_dir, img_name)
    print(img_name)
    # read the image
    img = cv2.imread(img_path)
    
    # resize the image
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)

    # make a prediction
    probs = model.predict(img)
    predictions = np.where(probs > 0.5, 1, 0)
    print(prediction)
    pred_class = categories[int((prediction[0]))]
    #img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    #cv2.imshow("img_name", img)
    print(f'Prediction for {img_name}: {pred_class}')

Normal (325).jpg
1/1 [==============================] - 0s 20ms/step
[[0.2713841]]
Prediction for Normal (325).jpg: Low Risk
HB   (141).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB   (141).jpg: Low Risk
PMI  (27).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for PMI  (27).jpg: Low Risk
Normal (260).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (260).jpg: Low Risk
HB (185).jpg
1/1 [==============================] - 0s 17ms/step
[[0.2713841]]
Prediction for HB (185).jpg: Low Risk
COVID  (8).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for COVID  (8).jpg: Low Risk
HB (490).jpg
1/1 [==============================] - 0s 17ms/step
[[0.2713841]]
Prediction for HB (490).jpg: Low Risk
HB   (116).jpg
1/1 [==============================] - 0s 20ms/step
[[0.2713841]]
Prediction for HB   (116).jpg: Low Risk
HB (428).jpg
1/1 [================

1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Binder1_Page_127.jpg: Low Risk
Normal (539).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (539).jpg: Low Risk
Normal (755).jpg
1/1 [==============================] - 0s 17ms/step
[[0.2713841]]
Prediction for Normal (755).jpg: Low Risk
Normal (305).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (305).jpg: Low Risk
Normal (217).jpg
1/1 [==============================] - 0s 17ms/step
[[0.2713841]]
Prediction for Normal (217).jpg: Low Risk
HB   (136).jpg
1/1 [==============================] - 0s 17ms/step
[[0.2713841]]
Prediction for HB   (136).jpg: Low Risk
Normal (352).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (352).jpg: Low Risk
Normal (702).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for Normal (702).jpg: Low Risk
HB (370).jpg
1/1 

1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (236).jpg: Low Risk
HB   (74).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB   (74).jpg: Low Risk
HB   (140).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB   (140).jpg: Low Risk
PMI  (26).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for PMI  (26).jpg: Low Risk
Normal (261).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (261).jpg: Low Risk
PMI (124).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for PMI (124).jpg: Low Risk
Normal (513).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (513).jpg: Low Risk
Normal (143).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (143).jpg: Low Risk
PMI (84).jpg
1/1 [=====================

1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (174).jpg: Low Risk
HB (442).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB (442).jpg: Low Risk
Normal (80).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (80).jpg: Low Risk
Binder1_Page_034.jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Binder1_Page_034.jpg: Low Risk
Normal (820).jpg
1/1 [==============================] - 0s 23ms/step
[[0.2713841]]
Prediction for Normal (820).jpg: Low Risk
Normal (123).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (123).jpg: Low Risk
HB (229).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB (229).jpg: Low Risk
Binder1_Page_222.jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for Binder1_Page_222.jpg: Low Risk
Binder1_Page_236.jp

1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB (248).jpg: Low Risk
Normal (284).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for Normal (284).jpg: Low Risk
HB (474).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB (474).jpg: Low Risk
HB (531).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB (531).jpg: Low Risk
COVID  (3).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for COVID  (3).jpg: Low Risk
HB (230).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB (230).jpg: Low Risk
Normal (706).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for Normal (706).jpg: Low Risk
HB   (165).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB   (165).jpg: Low Risk
Binder1_Page_198.jpg
1/1 [===========================

1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for Binder1_Page_207.jpg: Low Risk
HB (314).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB (314).jpg: Low Risk
Normal (40).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (40).jpg: Low Risk
PMI (184).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for PMI (184).jpg: Low Risk
HB   (30).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB   (30).jpg: Low Risk
Binder1_Page_011.jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for Binder1_Page_011.jpg: Low Risk
HB   (7).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB   (7).jpg: Low Risk
Binder1_Page_005.jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Binder1_Page_005.jpg: Low Risk
Normal (225).jpg
1/1 [=====

1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB (319).jpg: Low Risk
Normal (785).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for Normal (785).jpg: Low Risk
Binder1_Page_214.jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Binder1_Page_214.jpg: Low Risk
Binder1_Page_215.jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for Binder1_Page_215.jpg: Low Risk
HB   (11).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB   (11).jpg: Low Risk
Normal (36).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (36).jpg: Low Risk
HB (227).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB (227).jpg: Low Risk
HB   (46).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB   (46).jpg: Low Risk
HB   (172).jpg
1/1 [=======

1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB (298).jpg: Low Risk
HB   (122).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB   (122).jpg: Low Risk
HB (220).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB (220).jpg: Low Risk
Normal (346).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (346).jpg: Low Risk
Normal (31).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for Normal (31).jpg: Low Risk
PMI (55).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for PMI (55).jpg: Low Risk
PMI  (44).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for PMI  (44).jpg: Low Risk
HB   (16).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB   (16).jpg: Low Risk
Normal (595).jpg
1/1 [==============================] -

1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for PMI  (12).jpg: Low Risk
Binder1_Page_088.jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for Binder1_Page_088.jpg: Low Risk
HB (333).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB (333).jpg: Low Risk
Normal (605).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (605).jpg: Low Risk
HB   (56).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB   (56).jpg: Low Risk
HB   (162).jpg
1/1 [==============================] - 0s 15ms/step
[[0.2713841]]
Prediction for HB   (162).jpg: Low Risk
Normal (756).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for Normal (756).jpg: Low Risk
HB (325).jpg
1/1 [==============================] - 0s 16ms/step
[[0.2713841]]
Prediction for HB (325).jpg: Low Risk
Normal (428).jpg
1/1 [================

In [4]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('Dataset/ECG Images of COVID-19 Patients (250)/Binder1_Page_051.jpg')


## (1) Convert to gray, and threshold
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

## (2) Morph-op to remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

## (3) Find the max-area contour
cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
cnt = sorted(cnts, key=cv2.contourArea)[-1]

## (4) Crop and save it
x,y,w,h = cv2.boundingRect(cnt)
dst = img[y:y+h, x:x+w]
cv2.imwrite("001.png", dst)

True